In [76]:
# imports
import os
import time
import numpy as np
import pandas as pd

import pyroomacoustics as pra
from IPython.display import Audio

import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
from ament_index_python.packages import get_package_share_directory

import torch, torchaudio
import torchaudio.transforms as T
from torchaudio.models.decoder import download_pretrained_files, ctc_decoder

In [86]:
# params
filepath = '../../bags/e1_est_tuning'
actual_scene_labels = ['campus','courtyard','lab','lobby']

# sampling/acquisition
sample_rate = 44100
hop_size = 44100
frame_size = 44100 # publish 1 second every second
n_total_channels = 16
channel_indices_used = [0, 1, 2, 3, 4, 5, 6, 7]
n_channels_used = len(channel_indices_used)

speed_sound = 343.
n_sources = 2
n_fft = 512
f_min = 300
f_max = 8000
doa_dimension = 3
array_x_pos = [0.43, 0.43, -0.34, -0.34, 0., 0., 0., 0.]
array_y_pos = [-0.28, 0.28, 0.30, -0.30, -0.29, 0.29, -0.11, 0.11]
array_z_pos = [0.395, 0.395, 0.395, 0.395, 0.610, 0.610, 0.660, 0.660]
array_pos = np.array([array_x_pos,
                      array_y_pos]) # , array_z_pos
ssl_algo = 'MUSIC'
excess_front = int(np.ceil((n_fft-1)/2))
excess_back = int(np.floor((n_fft-1)/2))

# beamformer params

# VAD params
n_silent_frames = 2
trigger_time = 0.25
search_time = 0.25
allowed_gap = 0.25
pre_trigger_time = 0.25
min_voice_samples = 31200

# LM params
lexicon_package = 'ros_audition'
lexicon_file = 'config/rocog_lexicon.txt'

# Tunable parameters
vad_trigger_levels = [1., 5., 10.]
am_bundles = ['WAV2VEC2_ASR_BASE_960H','WAV2VEC2_ASR_LARGE_960H']
lm_weights = [-5.0,-1.0, -0.5, 0.0, 0.5, 1.0, 5.0]

In [87]:
# Create beamformer object, compute beam weights
source_angle = 0
bf = pra.Beamformer(array_pos, sample_rate, n_fft)
bf.far_field_weights(source_angle)

# Compute voice activity detector object
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Speech recognition objects
cmd_transcripts = {'advance': 'advance',
                   'attention': 'attention',
                   'follow-me': 'follow me',
                   'halt': 'halt',
                   'move-forward': "move forward",
                   'move-in-reverse': "move in reverse",
                   'rally': 'rally'}
                   

lm_files = download_pretrained_files("librispeech-4-gram")
WORD_SCORE = 0

resampler = T.Resample(sample_rate, bundle.sample_rate, dtype=torch.float16)
resampler = resampler.to(torch_device)

In [88]:
# Helper functions
def typename(topic_name, topic_types):
    for topic_type in topic_types:
        if topic_type.name == topic_name:
            return topic_type.type
    raise ValueError(f"topic {topic_name} not in bag")


In [89]:
# Initialize results
word_error_results = {}

for trigger_level in vad_trigger_levels:
    
    # Create VAD object, get name    
    vad = torchaudio.transforms.Vad(sample_rate, 
                                    trigger_level=trigger_level, 
                                    trigger_time=trigger_time,
                                    search_time=search_time,
                                    allowed_gap=allowed_gap,
                                    pre_trigger_time=pre_trigger_time) 
    vad.to(torch_device)
    vad_str = 'tl_%s' % (str(trigger_level).replace('.','')) # VAD param

    for bundle_name in am_bundles:
        # Create acoustic model, get name
        exec('bundle = torchaudio.pipelines.%s' % bundle_name)
        asr_model = bundle.get_model().to(torch_device)
        
        if bundle_name == 'WAV2VEC2_ASR_BASE_960H':
            acoustic_model_str = 'am_base'
        elif bundle_name == 'WAV2VEC2_ASR_LARGE_960H':
            acoustic_model_str = 'am_large'
        else:
            error('Not a valid bundle name')

        for lm_weight in lm_weights:
        
            # Create language model, get name
            beam_search_decoder = ctc_decoder(
                lexicon=os.path.join(get_package_share_directory(lexicon_package), lexicon_file),
                tokens=lm_files.tokens,
                lm=lm_files.lm,
                nbest=1,
                beam_size=50,
                lm_weight=lm_weight,
                word_score=WORD_SCORE,
            )
            lang_model_str = 'lm_%s' % str(lm_weight).replace('.','')
            
            model_str = "%s_%s_%s" % (vad_str, acoustic_model_str, lang_model_str)
            print("Evaluating %s" %model_str)
            
            word_error_results[model_str] = {}
            word_error_results[model_str]['overall'] = {}
            word_error_results[model_str]['overall']['total_words'] = 0
            word_error_results[model_str]['overall']['incorrect_words'] = 0
            
            # Iterate through each file
            # Traverse filepath
            for root, _, files in os.walk(filepath):
            
                # Examine each file
                for file in files:
                    
                    # If it is an mcap, examine
                    if os.path.splitext(file)[-1] == '.mcap':
            
                        # Get labels based on filepath
                        labels = os.path.normpath(root).split(os.sep)
                        command_actual = labels[-1]
                        cmd_mode_actual = labels[-2]
                        role_actual = labels[-3]
                        scene_actual = labels[-4]
                        iteration = os.path.splitext(file)[-2].split('_')[-1]
            
                        scene_actual_idx = actual_scene_labels.index(scene_actual)
            
                        if cmd_mode_actual not in ['gest_verb','verbal']:
                            continue
            
                        # Add new result dictionary keys
                        if scene_actual not in word_error_results[model_str].keys():
                            word_error_results[model_str][scene_actual] = {}
                            word_error_results[model_str][scene_actual]['total_words'] = 0
                            word_error_results[model_str][scene_actual]['incorrect_words'] = 0
                        
                        # Create reader object           
                        reader = rosbag2_py.SequentialReader()            
                        reader.open(
                            rosbag2_py.StorageOptions(uri=os.path.join(root,file), storage_id="mcap"),
                            rosbag2_py.ConverterOptions(
                                input_serialization_format="cdr", output_serialization_format="cdr"
                            ),
                        )
                        topic_types = reader.get_all_topics_and_types()
            
            
                        # RESET
                        # Create empty audio frame
                        audio_straight_ahead = np.array([])
                        
                        # Iterate through messages
                        while reader.has_next():
                            topic, data, timestamp = reader.read_next()
                            msg_type = get_message(typename(topic,topic_types))
            
                            if topic=='/audio_data':
                                audio_msg = deserialize_message(data,msg_type)
            
                                # start_time = time.time() # start clock
            
                                all_channels_chunk = torch.frombuffer(audio_msg.audio.data,dtype=torch.float16).view(-1,n_total_channels)
                                selected_channels_chunk = all_channels_chunk[:,channel_indices_used]
                              
                                bf.record(selected_channels_chunk.T, sample_rate)
                                signal = bf.process(FD=False).astype(np.float16)
                                                               
                                audio_straight_ahead_chunk = signal[excess_front:-excess_back] #.tobytes()
                                audio_straight_ahead = np.concatenate((audio_straight_ahead,audio_straight_ahead_chunk))
                                
                                                
                                # stop_time = time.time() - start_time
            
                        # Run voice activity detection on frame if it isn't empty
                        if (audio_straight_ahead.size > 0):
                            audio_straight_ahead_tensor = torch.from_numpy(audio_straight_ahead).half()
                            audio_straight_ahead_tensor.to(torch_device)
                            voice_tensor = vad.forward(audio_straight_ahead_tensor).to(torch_device)
                            
                            # run ASR
                            voice_tensor_resampled = resampler(voice_tensor).float()
                    
                            with torch.inference_mode():
                                emission, _ = asr_model(voice_tensor_resampled.view(1,-1))

                                if (torch.isnan(emission)==False).all().item():
                                    # CPU CTC beam search decoder
                                    beam_search_result = beam_search_decoder(emission.cpu())
                                    scores = [hyp.score for hyp in beam_search_result[0]]
                                    beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
                                
                                    # compare result to truth value, and scene
                                    actual_transcript = cmd_transcripts[command_actual].split()
                                    incorrect_words = torchaudio.functional.edit_distance(actual_transcript, beam_search_result[0][0].words)
                                    # print(actual_transcript)
                                    # print(beam_search_transcript)
                                    # print("numerator: %s" % incorrect_words)
                                    # print("denomenator: %s" % len(actual_transcript))
                                    # print()
                        
                                    # Increment results dictionary
                                    word_error_results[model_str][scene_actual]['total_words'] += len(actual_transcript)
                                    word_error_results[model_str]['overall']['total_words'] += len(actual_transcript)
                        
                                    word_error_results[model_str][scene_actual]['incorrect_words'] += incorrect_words
                                    word_error_results[model_str]['overall']['incorrect_words'] += incorrect_words

                                else:
                                    print("Problem with %s/%s - emission nan" % (root, file))
                        else:
                            print("Problem with %s/%s - no audio data" % (root, file))

Evaluating tl_10_am_base_lm_-50


[WARN] [1726176973.936810672] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176974.067668062] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Problem with ../../bags/e1_est_tuning/campus/supervisor/verbal/move-in-reverse/move-in-reverse_3.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/supervisor/verbal/move-in-reverse/move-in-reverse_1.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/supervisor/verbal/move-in-reverse/move-in-reverse_2.mcap - no audio data


[WARN] [1726176974.167242799] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176974.322830930] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176974.676218076] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176975.117078587] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176975.515328036] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176975.825387830] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176976.215685308] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176985.120451552] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726176985.922373124] [rosbag2_storage_mcap]: no

Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_3.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_1.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_2.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_4.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_0.mcap - no audio data


...

[WARN] [1726229262.130090775] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229262.411139057] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229262.756388047] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229263.090443228] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229263.333183783] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229263.662971503] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229271.476426891] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229272.114900112] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229272.746642944] [rosbag2_storage_mcap]: no

Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_3.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_1.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_2.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_4.mcap - no audio data
Problem with ../../bags/e1_est_tuning/campus/pedestrian/verbal/move-in-reverse/move-in-reverse_0.mcap - no audio data


[WARN] [1726229543.987938369] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229544.600565512] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229545.105287714] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Problem with ../../bags/e1_est_tuning/lobby/supervisor/verbal/move-in-reverse/move-in-reverse_2.mcap - emission nan


[WARN] [1726229545.502454329] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229561.691031514] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229561.959663715] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229562.202280161] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229562.388576831] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1726229562.600756504] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


In [90]:
# analyze word error results
results_columns = ['Model',
                   'Scene',
                   'Total Words',
                   'Incorrect Words',
                   'Word Error Rate']
results_df = pd.DataFrame(columns=results_columns)

idx=0
for model_key in word_error_results.keys():
    for scene_key in word_error_results[model_key].keys():
    
        results_df.loc[idx,'Model'] = model_key
        results_df.loc[idx,'Scene'] = scene_key
        results_df.loc[idx,'Total Words'] = word_error_results[model_key][scene_key]['total_words']
        results_df.loc[idx,'Incorrect Words'] = word_error_results[model_key][scene_key]['incorrect_words']
        results_df.loc[idx,'Word Error Rate'] = results_df.loc[idx,'Incorrect Words']/results_df.loc[idx,'Total Words']

        idx+=1

results_df

,Model,Scene,Total Words,Incorrect Words,Word Error Rate
0,tl_10_am_base_lm_-50,overall,1304,1513,1.160276
1,tl_10_am_base_lm_-50,lab,330,322,0.975758
2,tl_10_am_base_lm_-50,courtyard,344,400,1.162791
3,tl_10_am_base_lm_-50,campus,303,360,1.188119
4,tl_10_am_base_lm_-50,lobby,327,431,1.318043
...,...,...,...,...,...
205,tl_100_am_large_lm_50,overall,1304,1141,0.875
206,tl_100_am_large_lm_50,lab,330,288,0.872727
207,tl_100_am_large_lm_50,courtyard,344,293,0.851744
208,tl_100_am_large_lm_50,campus,303,251,0.828383


In [92]:
results_df.to_csv("../../results/speech_eval.csv")

In [59]:
Audio(data=audio_straight_ahead_tensor,rate=sample_rate)

/home/jd/mambaforge/envs/sit_int/lib/python3.10/site-packages/IPython/lib/display.py:188: RuntimeWarning: invalid value encountered in cast
  return scaled.astype("<h").tobytes(), nchan


In [60]:
Audio(data=voice_tensor.cpu(),rate=sample_rate)

In [61]:
Audio(data=voice_tensor_resampled.cpu(),rate=bundle.sample_rate)

In [70]:
(torch.isnan(emission)==False).all().item()

False